In [ ]:
# data files are exactly as received from MIRA Helsinki Study. This code does not write to these files.
data_lab = 'data/HUSLAB Data final_WithMixedVegans.txt'
data_intakes = 'data/muuttujat_analyysiin.txt'
data_thl = 'data/growth-curves.tsv'
data_questionnaire = 'data/data huoltajan tausta 171106.sav'
data_food_record = 'data/radata.txt'

In [ ]:
# animal proportion per food item collected for this thesis, method and sources on page 39 of thesis
data_food_animal_proportion = 'data/food-animal-proportion.csv'

In [ ]:
# global settings for graph output

import matplotlib.pyplot as plt
import seaborn as sns

scale=10
sns.set_theme(style='white',font_scale=2)

In [ ]:
# clean output directory every time
import shutil
shutil.rmtree('out')

from pathlib import Path
Path('out').mkdir(parents=True, exist_ok=True)

# Methods
## Design and participants
Figure 1
The participant flow in the original MIRA Helsinki Study in 2017 and in the present thesis. Dietary classification done based on the food record and the FFQ data. Two participants originally omitted due to difficulties to classify are included in the present study as omnivores.

# Results

In [ ]:
#declare common column names as variables
ldl = 'fP-Kol-LDL (mmol/l)'
tc = 'fP-Kol (mmol/l)'
serum_lipids = [
    tc,
    ldl, 
    'fP-Kol-HDL (mmol/l)', 
    'fP-Trigly (mmol/l)'
]

In [ ]:
# read in mira study data
import pandas as pd

lab_results = pd.read_csv(data_lab, sep='\t', decimal=",")
intakes = pd.read_csv(data_intakes, sep='\t')

df = lab_results.merge(intakes, on='ID')

# Only select subjects for whom we have an LDL lab result 
subjects = df[df[ldl].notna()]

In [ ]:
# Read in THL curves on finnish children

import numpy as np

curves = pd.read_csv(data_thl,sep='\t',decimal=',')
curves.columns = [c.lower() for c in curves.columns]

# no curve for children under two, fill in with a linear extrapolation

def fill_start_with_linear_extrapolation(s):
    i = s.first_valid_index()
    x1 = s.loc[i]
    x2 = s.loc[2*i]
    x0 = x1 - (x2-x1)
    return pd.Series(np.linspace(x0,x1,i)).append(s[i:])


for c in 'bmi_mean_m','bmi_sd_m','bmi_nu_m','bmi_mean_f','bmi_sd_f','bmi_nu_f':
    s = curves[c]
    s2 = fill_start_with_linear_extrapolation(s)
    curves.insert(
        curves.columns.get_loc(c)+1,
        c+'_filled',
        s2
    )

# BMISDS = ((BMIlaskettu / muBMI) ^ nuBMI – 1) / (nuBMI × sigmaBMI)
def translate_sex(s):
    if s in ('M','m'): return 'm'
    if s in ('N','n','F','f'): return 'f'
    return None

def bmi_sds(weight,height,age,sex):
    age = round(age,2)
    sex = translate_sex(sex)

    bmi = weight/height**2
    
    row = curves[curves.age==age].iloc[0]
    mu_bmi = row['bmi_mean_'+sex+'_filled']
    nu_bmi = row['bmi_nu_'+sex+'_filled']
    sigma_bmi = row['bmi_sd_'+sex+'_filled']
    
    bmi_sds = ((bmi/mu_bmi)**nu_bmi - 1) / (nu_bmi * sigma_bmi)
    
    return bmi_sds

if not 'bmi_sds' in subjects:
    bmi_sds = subjects.apply(
        lambda row: bmi_sds(
            row.Weight,
            row.Height/100,
            row.Bage,
            row.Sex
        ),
        axis=1
    )
    subjects.insert(12,'bmi_sds',bmi_sds)
    
def is_female(s):
    return s in ('N','n','F','f')

def is_male(s):
    return s in ('M','m')

def classify_bmi_sds(bmi_sds,sex):
    if is_female(sex):
        if bmi_sds < -2.2187: return 'Significantly underweight'
        if bmi_sds < -1.6482: return 'Underweight'
        if bmi_sds > 2.7600: return 'Severely Obese'
        if bmi_sds > 2.1065: return 'Obese'
        if bmi_sds > 1.1629: return 'Overweight'
        return 'Normal'
    if is_male(sex):
        if bmi_sds < -2.3456: return 'Significantly underweight'
        if bmi_sds < -1.8344: return 'Underweight'
        if bmi_sds > 2.3600: return 'Severely Obese'
        if bmi_sds > 1.7016: return 'Obese'
        if bmi_sds > 0.7784: return 'Overweight'
        return 'Normal'
    return None

if not 'bmi_class' in subjects:
    bmi_class = subjects.apply(
        lambda row: classify_bmi_sds(
            row.bmi_sds,
            row.Sex
        ),
        axis=1
    )

    subjects.insert(13,'bmi_class',bmi_class)

## Characteristics of the study participants
**Table 1** The study participant characteristics. Values expressed as medians (range) if not otherwise informed.  ¹Age of the child at the time of the blood sampling. ²BMI Standard Deviation (SD) Score is calculated from the Finnish population growth data (113).  ³Classification of overweight/obese and underweight based on the Finnish BMI-for-age percentile curves (113).

In [ ]:
df = subjects.groupby('Sex').count()[['ID']]
df.to_csv('out/table1-sex.csv')
df

In [ ]:
df = subjects[['Bage','bmi_sds',tc]].describe()
df.to_csv('out/table1-age-bmi-tc.csv')
df

In [ ]:
df = subjects.groupby('bmi_class').count()[['ID']]
df.to_csv('out/table1-bmi_class.csv')
df

**Table 2** The highest level of maternal education, expressed as either ongoing or completed studies. +lisää prosentit?

In [ ]:
def mothers_education(row):
    if row.v37=='1': return 0
    if row.v38=='1': return 1
    if row.v39=='1': return 2
    if row.v40=='1': return 3
    if row.v41=='1': return 4
    if row.v42=='1': return 5
    if row.v43=='1': return 6
    return None

def subject_mothers_education():
    df = pd.read_spss(
        data_questionnaire,
        usecols=['ID', 'v37', 'v38', 'v39', 'v40', 'v41', 'v42', 'v43']
    ).set_index('ID')
    for i,row in df.iterrows():
        df.at[i,'education_mother'] = mothers_education(row)
    return df[['education_mother']]

if not 'education_mother' in subjects:
    subjects = subjects.merge(
        subject_mothers_education(), 
        on='ID', 
        how='left'
    )
    
df = subjects.groupby('education_mother',dropna=False).count()[['ID']]
df.columns = ['num']
df['percent'] = df.num / df.num.sum() * 100.0
df = df.round(2)
df.to_csv('out/table2-education_mother.csv')
df

**Table 5.** The participants’ dietary intakes.

In [ ]:
subjects['ENER_kcal_per_d'] = subjects.ENERJ_per_d / 4.184

intake_e_pros = [
    'FAT_e_pros', 
    'FASAT_e_pros',
    'FAMS_e_pros',
    'FAPU_e_pros',
    'CHOL_per_MJ',
    'PROT_e_pros',
    'CHO_e_pros',
    'SUCS_e_pros',
    'FIBC_per_MJ',
    'SALT_per_MJ',
]
intake_per_d = [
    'ENERJ_per_d',
    'ENER_kcal_per_d',
    'FAT_e_pros', 
    'FASAT_e_pros',
    'FAMS_e_pros',
    'FAPU_e_pros',
    'CHOL_per_MJ',
    'PROT_e_pros',
    'CHO_e_pros',
    'SUCS_e_pros',
    'FIBC_per_MJ',
    'SALT_per_MJ',    
]
df = subjects[intake_e_pros+intake_per_d].describe().round(2)
df.to_csv('out/table3-intakes.csv')
df

## Food item scoring

In [ ]:
food_records = pd.read_csv(
    data_food_record, 
    sep='\t', 
    encoding='iso-8859-1')

food_animal_percent = pd.read_csv(
    data_food_animal_proportion, 
    index_col='code'
).drop(columns=['link'])

food_records = food_records.merge(
    food_animal_percent, 
    left_on='Code', 
    right_on='code', 
    how='left'
)

In [ ]:
# sanity check which food items with nonzero animal content appear for vegans

df = food_records.merge(
    subjects[['ID','Group4']], 
    on='ID', 
    how='left')
df = df[
    (df.animal_proportion > 0) 
    & (df.Group4 == 'Vegan')
]
df.groupby(['name','MaName']).count()['Group4']

**Table 4.** Number of food items within the deciles according to the proportion of animal origin content.

In [ ]:
df = food_animal_percent.groupby('animal_decile').count()
df.to_csv('out/table4-food_items_per_animal_decile.csv')
df

## ASE intakes among the dietary categories
**Figure 2** Histogram showing the distribution of ASE proportions by the diet classification (grouping based on the food records and FFQs). *Vegetarian group included lactovegetarians, lacto-ovo-vegetarians and pescovegetarians.

In [ ]:
# calculate ASE proportion
r_animal = 'r_animal_source_energy'
r_animal_label = 'Animal source energy proportion'

e = food_records[['ID','ENERJ','animal_proportion']]
e = e.assign(ENERJ_animal=(e.ENERJ * e.animal_proportion)).drop(columns=['animal_proportion'])

#energy per subject
eps = e.groupby(['ID']).sum()
eps = eps.assign(r_animal_source_energy=(eps.ENERJ_animal / eps.ENERJ))

if not r_animal in subjects:
    subjects = subjects.merge(eps, on='ID')

In [ ]:
# classify diets
g6_map = {
    'Pesco-vegetarian': 'Vegetarian',
    'Vegan': 'Vegan',
    'Control': 'Omnivore',
    'Control (vegan in daycare)': 'Omnivore',
    'Vegetarian': 'Vegetarian'
}
diet_class = 'diet classification'
diet_classes = ['Vegan','Vegetarian','Omnivore']

if not diet_class in subjects:
    subjects.insert(6,diet_class,subjects.Group4.map(g6_map))

In [ ]:
plt.close()
fg = sns.displot(
    subjects,
    x=r_animal,
    binwidth=0.05,
    hue=diet_class,
    hue_order=diet_classes,
    multiple='stack',
    height=scale)
fg.axes[0,0].set_xlabel(r_animal_label)
plt.savefig('out/figure2-ASE-histogram.png')

## Animal ratio and dietary intakes
Table 4
Pearson correlation coefficients for correlations between ASE proportion and dietary intakes. *Significance of the p-value corrected for multiple analyses using the Benjamini-Hochberg procedure, FDR 0.05.

In [ ]:
from scipy.stats import pearsonr
from statsmodels.stats.multitest import multipletests

def correlation_values(xs,ys):
    table = []
    for x in xs:
        for y in ys:
            df = subjects[[x,y]].dropna()
            pr, pp = pearsonr(df[x],df[y])
            table.append([x,y,pr,pp])

    df = pd.DataFrame(table,columns=['x','y','pearson_r','pearson_p'])
    df['fdr_bh_0_05'] = multipletests(df['pearson_p'], alpha=0.05, method='fdr_bh')[0]
    df = df.sort_values(by='pearson_p')
    df = df.round(3)
    return df

In [ ]:
df = correlation_values([r_animal],intake_e_pros)
df.to_csv('out/table5-correlations-ASE-intake_e_pros.csv')
df

In [ ]:
df = correlation_values([r_animal],intake_per_d)
df.to_csv('out/table5-correlations-ASE-intake_per_d.csv')
df

In [ ]:
def regplot_r_animal(ax,y,y_label):
    ax.set_xlim(-0.01, subjects.r_animal_source_energy.max()+0.01)
    sns.regplot(ax=ax, x=r_animal, y=y, scatter=False, data=subjects)
    sns.scatterplot(
        ax=ax, 
        x=r_animal, 
        y=y, 
        hue=diet_class, 
        hue_order=diet_classes, 
        data=subjects, 
        s=30*scale,
        legend=False
    )
    ax.set_xlabel(r_animal_label)
    ax.set_ylabel(y_label)


def fig_of_regplots(ys,cols=2):

    plt.close()
    sns.set_theme(style='white',font_scale=2)
    
    rows = int(len(ys)/cols)+(len(ys)%cols > 0)
    h = rows*scale
    if rows > 1: h+=2
    w = cols*scale+2
    


    fig, axs = plt.subplots(rows,cols,figsize=(w,h))
        
    if rows > 1:
        axs_flat = [ax for row in axs for ax in row]
    else:
        axs_flat = axs

    l = list(zip(ys.keys(),ys.values(),axs_flat))
    
    for t in l:
        regplot_r_animal(t[2],t[0],t[1])

    for ax in axs_flat[len(ys):]:
        fig.delaxes(ax)
        
    return fig, axs

Figures 3 a), b) and c)
Scatter plots showing the correlations between animal source energy and dietary intakes of a) cholesterol (mg per MJ/d), b) saturated fatty acids (E%) and c) polyunsaturated fatty acids (E%). In Figure 3 b) the participants achieving a ASE proportion of 0 (n=7) are piled up on top of each other in the left corner (0,0). Pearson correlation coefficients are displayed, significance of p-values expressed after correction of multiple analyses with the Benjamini-Hochberg procedure. The shaded light blue area represents the 95% confidence interval for the regression. In Figure 3a), the dashed red line represents the official dietary recommendation for children, which for SAFA intake is <10E%. Correspondingly, in Figure 3b) the dashed red line represents the lower reference intake for PUFA in the official dietary recommendations (>5 E%).

In [ ]:
ys = {
    'FASAT_e_pros': 'Saturated Fat (E%)',
    'FAPU_e_pros': 'Polyunsaturated Fat (E%)',
    'CHOL_per_MJ': 'Cholesterol (per MJ)',
}

fig, axs = fig_of_regplots(ys)

axs[0][0].axhline(10,ls='--',color='r') #FASAT high

axs[0][1].axhline(5,ls='--',color='r') # FAPU low
axs[0][1].axhline(10,ls='--',color='r') # FAPU high


plt.savefig('out/figure3-scatter-ASE-SAFA-PUFA-TC.png')

plt.show()

Figure 4a) and b).
Scatter plots of the correlations between animal source energy and dietary intakes of a) fibre (g per MJ) and b) protein (E%). In Figure 4a), the official dietary recommendation for fibre intake for over 2-year old children (2–3 g/MJ) is indicated with dashed red lines. Similarly, in Figure 4b) the dietary recommendation for protein intake (10–20 E%) in children is displayed with the red dashed lines. In addition, Figures present Pearson correlation coefficients and significance after correction for multiple analyses with Benjamini-Hochberg procedure. The shaded area represents the 95% confidence interval for the regression. 

In [ ]:
ys = {
    'FIBC_per_MJ': 'Fiber (per MJ)',
    'PROT_e_pros': 'Protein (E%)',
}

fig, axs = fig_of_regplots(ys)

axs[0].axhline(2,ls='--',color='r') # FIBC low
axs[0].axhline(3,ls='--',color='r') # FIBC high

axs[1].axhline(10,ls='--',color='r') #prot low
axs[1].axhline(20,ls='--',color='r') #prot high

plt.savefig('out/figure4-scatter-ASE-FIBC-PROT.png')

plt.show()

**Figure 5.** Stripplot of LDL-C concentrations in dietary groups

In [ ]:
plt.close()
fig, ax = plt.subplots(figsize=(scale,scale))
ax = sns.stripplot(
    x=diet_class,
    order=diet_classes,
    y=ldl,
    data=subjects,
    s=15,
    linewidth=1,
    edgecolor="white"
)
ax.set_xlabel('Diet classification')
ax.set_ylabel('LDL-C concentration (mmol/l)')

plt.savefig('out/figure5-stripplot-diet-LDL.png')

**Figure 6.**
The scatter plots of ASE proportion and plasma concentrations of A) total cholesterol, B) LDL-cholesterol, C) HDL-cholesterol D)Triglyceride. The shaded light blue area represents the 95% confidence interval for the regression. In Figure 5 a), some participants achieving a ASE proportion of 0 are on top of each other in data points (0,3 and 0,2.7 tms?) 

In [ ]:
serum_lipid_labels=['TC concentration','LDL-C concentration','HDL-C concentration','Triglyceride concetration']

fig, axs = fig_of_regplots(dict(zip(serum_lipids,serum_lipid_labels)))

plt.savefig('out/figure6-scatter-ASE-serum_lipids.png')

plt.show()

## Multiple regression model to explain plasma LDL-C concentration


In [ ]:
import statsmodels.api as sm

def ols_result(xs,ys):
    df = subjects[xs+ys].dropna()
    
    X = df[xs]
    Y = df[ys]
    
    X = sm.add_constant(X)
    
    return sm.OLS(Y, X).fit()

## Animal source energy proportion as a predictor of plasma LDL-cholesterol
Table 5
Linear regression model for animal source energy as a predictor of LDL-cholesterol concentrations. The unadjusted model.

In [ ]:
summary = ols_result([r_animal],[ldl]).summary()

file = open('out/table5-OLS-ASE-LDL.html', 'w')
file.write(summary.as_html())
file.close()
summary

**Table 6.** The multiple linear regression model to predict plasma LDL-cholesterol concentrations with animal source energy, child’s sex and maternal education. 

In [ ]:
subjects.loc[subjects.Sex == 'N', 'sex_dummy'] = 0
subjects.loc[subjects.Sex == 'M', 'sex_dummy'] = 1

summary = ols_result([r_animal,'sex_dummy','education_mother'],[ldl]).summary()

file = open('out/table6-OLS-ASE_sex_education-LDL.html', 'w')
file.write(summary.as_html())
file.close()
summary

## Biomarkers of cholesterol metabolism and bile acid synthesis

**Table 7.** The Pearson correlation coefficients between animal source energy and plasma cholesterol absorption and synthesis biomarkers and the plasma bile acid synthesis biomarker (7-OH-4-cholesten-3-one). The cholesterol synthesis and absorption markers expressed as their ratios to plasma TC concentrations. 

In [ ]:
chol_absorption_markers = [
    'Cholestanol',
    'Campesterol',
    'Sitosterol',
    'Avenasterol',
]
chol_synthesis_markers = [
    'Cholestenol',
    'Desmosterol',
    'Lathosterol',
    'Squalene',
]
subjects['r_campesterol_to_cholestanol'] = subjects.Campesterol / subjects.Cholestanol

df = correlation_values(
    [r_animal],
    chol_absorption_markers+chol_synthesis_markers+['7-OH-4-cholesten-3-one','r_campesterol_to_cholestanol']+serum_lipids)
df.to_csv('out/table7-correlations-ASE-cholesterol_markers.csv')
df

**Figure 7**. Correlations between ASE proportion and the serum biomarkers of cholesterol absorption, a) cholestanol, b) campesterol, c) sitosterol and d) avenasterol, expressed as their ratios to total cholesterol (100 x μg/mg).

In [ ]:
ys = dict(zip(chol_absorption_markers,chol_absorption_markers))

fig, axs = fig_of_regplots(ys)

plt.savefig('out/figure7-scatter-ASE-chol_absorption_markers.png')

plt.show()

**Figure 8.** Correlations between ASE proportion and the serum biomarkers of cholesterol synthesis, a) cholestenol, b) desmosterol, c) lathosterol and d) squalene, expressed as their ratios to total cholesterol (100 x μg/mg).

In [ ]:
ys = dict(zip(chol_synthesis_markers,chol_synthesis_markers))

fig, axs = fig_of_regplots(ys)

plt.savefig('out/figure8-scatter-ASE-chol_synthesis_markers.png')

plt.show()

**Figure 9.** Correlation of ASE proportion and serum concentration of bile acid synthesis marker 7α-OH-4-cholesten-3-one (μmol/l).

In [ ]:
plt.close()

fig, ax = plt.subplots(figsize=(scale,scale))

regplot_r_animal(ax,'7-OH-4-cholesten-3-one','7-OH-4-cholesten-3-one')

plt.savefig('out/figure9-scatter-ASE-bile_acid_synthesis_marker.png')

plt.show()

## Bile acid composition

In [ ]:
glycine_conjugated_bile_acids = ['GCDCA','GCA','GLCA','GUDCA','GDCA']
taurine_conjugated_bile_acids = ['TUDCA','TCA','TDCA','TLCA','TCDCA']
unconjugated_bile_acids = ['UDCA','HDCA','CDCA','DCA','LCA','CA']
all_bile_acids = unconjugated_bile_acids + glycine_conjugated_bile_acids + taurine_conjugated_bile_acids

subjects = subjects.assign(
    total_bile_acids=subjects[all_bile_acids].sum(axis=1))
subjects = subjects.assign(
    total_glycine_conjugated_bile_acids = subjects[glycine_conjugated_bile_acids].sum(axis=1))
subjects = subjects.assign(
    total_taurine_conjugated_bile_acids = subjects[taurine_conjugated_bile_acids].sum(axis=1))
subjects = subjects.assign(
    conjugated_bile_acid_ratio = (
        subjects.total_taurine_conjugated_bile_acids / subjects.total_glycine_conjugated_bile_acids))

bile_acid_group_totals = [
    'total_bile_acids',
    'total_glycine_conjugated_bile_acids', 
    'total_taurine_conjugated_bile_acids',
    'conjugated_bile_acid_ratio'
]


**Table 8.** Pearson correlation coefficients between animal source energy intake and the plasma bile acid variables. Significance given after correcting for multiple analysis with the Benjamini-Hochberg procedure, FDR 0.05.

In [ ]:
df = correlation_values(
    [r_animal],
    all_bile_acids+bile_acid_group_totals
)
df.to_csv('out/table8-correlations-ASE-bile_acids.csv')
df

# Supplements